In [ ]:
import numpy as np
import random
from shapely.geometry import Polygon
import rtsvg
rt  = rtsvg.RACETrack()
def randomCircleConfig(n_circles=15, w=128, h=128, r_min=3.0, r_max=15.0, d_intra_circle_min=3.0):
    attempts, circles = 0, []
    while len(circles) < n_circles:
        r = random.uniform(r_min, r_max)
        x = random.uniform(r+d_intra_circle_min, w-r-d_intra_circle_min)
        y = random.uniform(r+d_intra_circle_min, h-r-d_intra_circle_min)
        overlaps = False
        for c in circles:
            d = rt.segmentLength(((x,y), c))
            if c[2] + r + d_intra_circle_min > d : overlaps = True
        if overlaps == False: circles.append((x,y,r))
        attempts += 1
        if attempts > 1000: raise Exception('Failed to generate circles')
    return circles

In [ ]:
#circles = [(25.0,25.0,20.0),(80.0,80.0,30.0),(20.0,100.0,10.0),(25.0,60.0,5.0),(60.0,30.0,5.0),(48.0,48.0,6.0),(40.0,90.0,6.0)]
circles = randomCircleConfig()
cell_map, tri_list, V = rt.laguerreVoronoi2D(circles)
x_min, y_min, x_max, y_max = 1e9, 1e9, -1e9, -1e9
for _pair_ in V:
    x,y = _pair_
    x_min, x_max = min(x_min, x), max(x_max, x)
    y_min, y_max = min(y_min, y), max(y_max, y)
for i in range(len(circles)):
    x,y,r = circles[i]
    x_min, x_max = min(x_min, x-r), max(x_max, x+r)
    y_min, y_max = min(y_min, y-r), max(y_max, y+r)
print(f'x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}')
_border_ = 10.0
x_min, y_min, x_max, y_max = x_min - _border_, y_min - _border_, x_max + _border_, y_max + _border_
#svg_hdr = f'<svg x="0" y="0" width="768" height="768" viewbox="{x_min} {y_min} {x_max-x_min} {y_max-y_min}"><rect x="{x_min}" y="{y_min}" width="{x_max-x_min}" height="{y_max-y_min}" fill="white" />'
svg_hdr = f'<svg x="0" y="0" width="768" height="768" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr = '</svg>'
svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.6"/>')
    svg_circles.append(rt.svgText(str(i), circles[i][0], circles[i][1], txt_h=4.0, anchor='middle'))
# rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])
svg_v, svg_fmto = [], []
for _pair_ in V:
    x,y = _pair_
    svg_v.append(f'<circle cx="{x}" cy="{y}" r="0.5" fill="#000000" stroke="none" />')

_light_gray_ = '#d0d0d0'
for _tri_ in tri_list:
    line_0 = f'<line x1="{circles[_tri_[0]][0]}" y1="{circles[_tri_[0]][1]}" x2="{circles[_tri_[1]][0]}" y2="{circles[_tri_[1]][1]}" stroke="{_light_gray_}" stroke-width="0.2" />'
    line_1 = f'<line x1="{circles[_tri_[1]][0]}" y1="{circles[_tri_[1]][1]}" x2="{circles[_tri_[2]][0]}" y2="{circles[_tri_[2]][1]}" stroke="{_light_gray_}" stroke-width="0.2" />'
    line_2 = f'<line x1="{circles[_tri_[2]][0]}" y1="{circles[_tri_[2]][1]}" x2="{circles[_tri_[0]][0]}" y2="{circles[_tri_[0]][1]}" stroke="{_light_gray_}" stroke-width="0.2" />'
    svg_v.append(line_0), svg_v.append(line_1), svg_v.append(line_2)
    svg_fmto.append(line_0), svg_fmto.append(line_1), svg_fmto.append(line_2)

for i in range(len(cell_map)):
    for j in range(len(cell_map[i])):
        fm_to = cell_map[i][j][0]
        if fm_to[0] == -1 or fm_to[1] == -1:
            if   fm_to[0] == -1:
                _to_ = fm_to[1]
                xy   = V[_to_]
                uv   = cell_map[i][j][1][1]
                l    = cell_map[i][j][1][3]
            elif fm_to[1] == -1:
                _fm_ = fm_to[0]
                xy   = V[_fm_]
                uv   = cell_map[i][j][1][1]
                l    = cell_map[i][j][1][3]
            if l is None: l = 100.0
            svg_fmto.append(f'<line x1="{xy[0]}" y1="{xy[1]}" x2="{xy[0]+uv[0]*l}" y2="{xy[1]+uv[1]*l}" stroke="red" stroke-width="0.4" />')    
        else:
            _fm_, _to_ = fm_to[0], fm_to[1]
            svg_fmto.append(f'<line x1="{V[_fm_][0]}" y1="{V[_fm_][1]}" x2="{V[_to_][0]}" y2="{V[_to_][1]}" stroke="black" stroke-width="0.2" />')
for xy in V:
    svg_fmto.append(f'<circle cx="{xy[0]}" cy="{xy[1]}" r="{0.5+random.random()}" fill="none" stroke="#0000ff" stroke-width="0.1" />')
#rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_v) + svg_ftr,
#         svg_hdr + ''.join(svg_circles) + ''.join(svg_fmto) + svg_ftr], spacer=10)

In [ ]:
x_min, y_min, x_max, y_max = circles[0][0], circles[0][1], circles[0][0], circles[0][1]
for circle in circles:
    x_min, x_max = min(x_min, circle[0]-circle[2]), max(x_max, circle[0]+circle[2])
    y_min, y_max = min(y_min, circle[1]-circle[2]), max(y_max, circle[1]+circle[2])
_border_ = 2.0
x_min, y_min, x_max, y_max = x_min-_border_, y_min-_border_, x_max+_border_, y_max+_border_
svg_border = [f'<rect x="{x_min}" y="{y_min}" width="{x_max-x_min}" height="{y_max-y_min}" stroke="#e0e0e0" stroke-width="2.0" fill="none" />']

epsilon = 1e-6
def inBounds(_xy_):           return (x_min-epsilon) <= _xy_[0] <= (x_max+epsilon) and (y_min-epsilon) <= _xy_[1] <= (y_max+epsilon)
def bothInBounds(_segment_):  return inBounds(_segment_[0]) and inBounds(_segment_[1])
_segments_, xy_to_cell_map_is, xys_to_dedupe = [], {}, set()

# Updates the xy to circle list
def updateXYToCellMap(_segment_, _cell_i_):
    for i in range(2):
        _xy_ = (_segment_[i][0], _segment_[i][1])
        if _xy_ not in xy_to_cell_map_is: xy_to_cell_map_is[_xy_] = set()
        xy_to_cell_map_is[_xy_].add(_cell_i_)

# Use the cell map to create the segments & associate vertices to circles
for i in range(len(cell_map)):
    for j in range(len(cell_map[i])):        
        fm_to = cell_map[i][j][0]
        # If either are negative one, then it's a ray
        if fm_to[0] == -1 or fm_to[1] == -1:
            if   fm_to[0] == -1:
                _to_ = fm_to[1]
                xy   = V[_to_]
                uv   = cell_map[i][j][1][1]
                l    = cell_map[i][j][1][3]
            elif fm_to[1] == -1:
                _fm_ = fm_to[0]
                xy   = V[_fm_]
                uv   = cell_map[i][j][1][1]
                l    = cell_map[i][j][1][3]
            if   l is None: l =  2.0 * max(x_max - x_min, y_max - y_min) # maximum length of the ray required (actually a hack)
            elif l == 0:    l = -2.0 * max(x_max - x_min, y_max - y_min) # ... this corrects the problem of fully connecting circles to their edge points
            _segment_ = (xy, (xy[0]+uv[0]*l, xy[1]+uv[1]*l))
        # Else it's a segment (but not necessarily a segment that begins/ends within the boundary)
        else:
            _fm_, _to_ = fm_to[0], fm_to[1]
            _segment_ = (V[_fm_], V[_to_])
        
        # Check if the segment intersects the bounds
        _left_   = rt.segmentsIntersect(_segment_, ((x_min, y_min), (x_min, y_max)))
        _right_  = rt.segmentsIntersect(_segment_, ((x_max, y_min), (x_max, y_max)))
        _top_    = rt.segmentsIntersect(_segment_, ((x_min, y_max), (x_max, y_max)))
        _bottom_ = rt.segmentsIntersect(_segment_, ((x_min, y_min), (x_max, y_min)))

        # If it does intersect the bounds, clip it there (it's possible it intersects the bounds twice...)
        if _left_[0] or _right_[0] or _top_[0] or _bottom_[0]:
            intersections_found = 0
            if _left_  [0]: x, y, intersections_found = _left_  [1], _left_  [2], intersections_found + 1
            if _right_ [0]: x, y, intersections_found = _right_ [1], _right_ [2], intersections_found + 1
            if _top_   [0]: x, y, intersections_found = _top_   [1], _top_   [2], intersections_found + 1
            if _bottom_[0]: x, y, intersections_found = _bottom_[1], _bottom_[2], intersections_found + 1
            if    inBounds(_segment_[0]): xy = _segment_[0]
            else:                         xy = _segment_[1]
            if   intersections_found == 1:
                _segment_ = (xy, (x, y))
                if bothInBounds(_segment_):
                    _segments_.append(_segment_), updateXYToCellMap(_segment_, i)
                    xys_to_dedupe.add((x,y))
                else:
                    ...
            elif intersections_found == 2:
                if   _left_[0]:  x_other, y_other = _left_[1],  _left_[2]
                elif _right_[0]: x_other, y_other = _right_[1], _right_[2]
                elif _top_[0]:   x_other, y_other = _top_[1],   _top_[2]
                else: raise Exception(f'two intersections_found but upon further review the other one can\'t be found')
                _segment_ = ((x,y), (x_other, y_other))
                xys_to_dedupe.add((x,y)), xys_to_dedupe.add((x_other, y_other))
                _segments_.append(_segment_), updateXYToCellMap(_segment_, i)
            else:                        raise Exception(f'intersections_found={intersections_found} (should only be one or two)')
        # Otherwise, as long as both points are in bounds... then it can be added
        # ...  it's possible that the segment begins and ends out of bounds (and doesn't touch the boundary)
        else:
            if bothInBounds(_segment_):
                _segments_.append(_segment_), updateXYToCellMap(_segment_, i)
            else:
                ...

# Determine if the xy vertices need to be deduped & create the lookup table in the process
xys_to_dedupe = list(xys_to_dedupe)
dedupe_lu     = {}
epsilon       = 0.001
for i in range(len(xys_to_dedupe)):
    xy = xys_to_dedupe[i]
    if xy in dedupe_lu: continue
    for j in range(i+1, len(xys_to_dedupe)):
        other = xys_to_dedupe[j]
        l = rt.segmentLength((xy, other))
        if l < epsilon: dedupe_lu[other] = xy
    dedupe_lu[xy] = xy

# Dedupe the xy vertices
if len(dedupe_lu.keys()) != len(set(dedupe_lu.values())):
    new_segments = []
    for _segment_ in _segments_:
        xy0, xy1 = (_segment_[0][0], _segment_[0][1]), (_segment_[1][0], _segment_[1][1])
        if xy0 in dedupe_lu: xy0 = dedupe_lu[xy0]
        if xy1 in dedupe_lu: xy1 = dedupe_lu[xy1]
        new_segments.append((xy0,xy1))
    _segments_ = new_segments
    for _xy_ in dedupe_lu.keys():
        if _xy_ == dedupe_lu[_xy_]: continue
        xy_to_cell_map_is[dedupe_lu[_xy_]] |= xy_to_cell_map_is[_xy_]
        del xy_to_cell_map_is[_xy_]

# Remove any duplicate segments
deduped_segments, segments_seen = [], set()
for _segment_ in _segments_:
    if _segment_ in segments_seen: continue
    segments_seen.add(_segment_), segments_seen.add((_segment_[1], _segment_[0]))
    deduped_segments.append(_segment_)
print(f'{len(_segments_)=} {len(deduped_segments)=}')
_segments_ = deduped_segments

# Add segments for the borders
corner_bl = (np.float64(x_min), np.float64(y_min))
corner_br = (np.float64(x_max), np.float64(y_min))
corner_tl = (np.float64(x_min), np.float64(y_max))
corner_tr = (np.float64(x_max), np.float64(y_max))
corners   = [corner_bl, corner_br, corner_tr, corner_tl]

# Associate the corners with a circle
for _corner_ in corners:
    possible_circles = set()
    for circle_i in range(len(circles)):
        _segment_ = (_corner_, circles[circle_i][:2])
        segment_intersection = False
        for _other_ in _segments_:
            if rt.segmentsIntersect(_segment_, _other_)[0]: 
                segment_intersection = True
                break
        if segment_intersection == False: possible_circles.add(circle_i)
    if len(possible_circles) == 1:
        circle_i = list(possible_circles)[0]
        if _corner_ not in xy_to_cell_map_is: xy_to_cell_map_is[_corner_] = set()
        xy_to_cell_map_is[_corner_].add(circle_i)
    #else: raise Exception(f'Failed to find a circle for {_corner_} {possible_circles}')

# Add segments for the borders
for _y_ in [y_min,y_max]:
    for _xy_ in set(dedupe_lu.values()):
        to_arrange = set([(np.float64(x_min),np.float64(_y_)),(np.float64(x_max),np.float64(_y_))])
        if (_y_+epsilon) >= _xy_[1] >= (_y_-epsilon): to_arrange.add(_xy_)
        to_arrange = sorted(list(to_arrange))
        for i in range(len(to_arrange)-1):
            _segment_ = (to_arrange[i], to_arrange[i+1])
            if _segment_ in _segments_: continue
            _segments_.append(_segment_)
for _x_ in [x_min,x_max]:
    for _xy_ in set(dedupe_lu.values()):
        to_arrange = set([(np.float64(_x_),np.float64(y_min)),(np.float64(_x_),np.float64(y_max))])
        if (_x_+epsilon) >= _xy_[0] >= (_x_-epsilon): to_arrange.add(_xy_)
        to_arrange = sorted(list(to_arrange), key=lambda x: x[1])
        for i in range(len(to_arrange)-1):
            _segment_ = (to_arrange[i], to_arrange[i+1])
            if _segment_ in _segments_: continue
            _segments_.append(_segment_)
        
# Render the segments
xys_seen = set()
for _segment_ in _segments_:
    svg_border.append(f'<line x1="{_segment_[0][0]}" y1="{_segment_[0][1]}" x2="{_segment_[1][0]}" y2="{_segment_[1][1]}" stroke="black" stroke-width="0.1" />')
    _xy0_ = (_segment_[0][0], _segment_[0][1])
    _xy1_ = (_segment_[1][0], _segment_[1][1])
    xys_seen.add(_xy0_), xys_seen.add(_xy1_)
for _xy_ in xys_seen:
    svg_border.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5 + 1.5*random.random()}" fill="none" stroke="#000000" stroke-width="0.1" />')
for _xy_ in xy_to_cell_map_is.keys():
    if _xy_ not in xys_seen: continue
    for _cell_i_ in xy_to_cell_map_is[_xy_]:
        _color_ = rt.co_mgr.getColor(_cell_i_)
        svg_border.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{circles[_cell_i_][0]}" y2="{circles[_cell_i_][1]}" stroke="{_color_}" stroke-width="0.25" stroke-dasharray=".6 .8 .2"/>')
for _xy_ in xy_to_cell_map_is:
    if _xy_ != corner_bl and _xy_ != corner_br and _xy_ != corner_tl and _xy_ != corner_tr:
        if len(xy_to_cell_map_is[_xy_]) == 1:
            svg_border.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="2.0" fill="none" stroke="#ff0000" stroke-width="0.5" />')
# for i in range(len(V)): svg_border.append(rt.svgText(str(i), V[i][0], V[i][1], txt_h=4.0, anchor='middle'))

print(f'{len(xys_seen)=} {len(xys_to_dedupe)=} {len(dedupe_lu.keys())=} {len(dedupe_lu.values())=} {len(xy_to_cell_map_is.keys())=}')
rt.tile([svg_hdr + ''.join(svg_border)  + ''.join(svg_circles) + svg_ftr])

In [ ]:
from math import atan2
circle_i_to_xys = {}
for i in range(len(circles)): circle_i_to_xys[i] = set()
for _xy_ in xy_to_cell_map_is.keys():
    for _cell_i_ in xy_to_cell_map_is[_xy_]:
        circle_i_to_xys[_cell_i_].add(_xy_)
circle_i_to_segments = {}
for i in range(len(circles)): circle_i_to_segments[i] = set()
xy_to_segments = {}
for _segment_ in _segments_:
    xy0, xy1 = _segment_[0], _segment_[1]
    if xy0 not in xy_to_segments: xy_to_segments[xy0] = set()
    if xy1 not in xy_to_segments: xy_to_segments[xy1] = set()
    xy_to_segments[xy0].add(_segment_), xy_to_segments[xy1].add(_segment_)
for i in range(len(circles)):
    for _xy_ in circle_i_to_xys[i]:
        for _segment_ in xy_to_segments[_xy_]:
            if _segment_[0] in circle_i_to_xys[i] and _segment_[1] in circle_i_to_xys[i]:
                circle_i_to_segments[i].add(_segment_)
def orderCounterClockwise(segs):
    xy_to_segs = {}
    for _segment_ in segs:
        for i in range(2):
            _xy_ = (_segment_[i][0], _segment_[i][1])
            if _xy_ not in xy_to_segs: xy_to_segs[_xy_] = set()
            xy_to_segs[_xy_].add(_segment_)
    x_sum, y_sum, samples = 0.0, 0.0, 0
    for _xy_ in xy_to_segs.keys():
        x_sum += _xy_[0]
        y_sum += _xy_[1]
        samples += 1
    center       = (x_sum/samples, y_sum/samples)
    angle_sorter = []
    for _xy_ in xy_to_segs.keys():
        angle = atan2(_xy_[1]-center[1], _xy_[0]-center[0])
        angle_sorter.append((angle, _xy_))
    angle_sorter = sorted(angle_sorter, reverse=True)
    in_order = []
    for angle, _xy_ in angle_sorter: in_order.append(_xy_)
    return in_order

svg_cells = []
for i in range(len(circles)):
    _color_   = rt.co_mgr.getColor(i)
    _ordered_ = orderCounterClockwise(circle_i_to_segments[i])
    d = f'M {_ordered_[0][0]} {_ordered_[0][1]} '
    for j in range(1,len(_ordered_)): d += f'L {_ordered_[j][0]} {_ordered_[j][1]} '
    d += 'Z'
    svg_cells.append(f'<path d="{d}" fill="{_color_}" stroke="none" stroke-width="0.5" />')        
rt.tile([svg_hdr + '<g opacity="0.5">' + ''.join(svg_cells) + '</g>' + ''.join(svg_circles) + svg_ftr])

In [ ]:
cells = rt.laguerreVoronoi(circles)
from_framework_svg = []
for i in range(len(cells)):
    _cell_ = cells[i]
    d = f'M {_cell_[0][0]} {_cell_[0][1]} '
    for j in range(1,len(_cell_)): d += f'L {_cell_[j][0]} {_cell_[j][1]} '
    d += 'Z'
    from_framework_svg.append(f'<path d="{d}" fill="{rt.co_mgr.getColor(i)}" stroke="none" stroke-width="0.5" />')        
rt.tile([svg_hdr + '<g opacity="0.5">' + ''.join(from_framework_svg) + '</g>' + ''.join(svg_circles) + svg_ftr])

In [ ]:
three_circles = [(60,60,4),(70,70,5),(75,50,8)]
_svg_ = [f'<svg x="0" y="0" width="512" height="512" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />']
for i in range(len(three_circles)):
    _svg_.append(f'<circle cx="{three_circles[i][0]}" cy="{three_circles[i][1]}" r="{three_circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.6"/>')
bounds = [(64,5),(5,64),(64,123),(123,64)]
d = f'M {bounds[0][0]} {bounds[0][1]} '
for i in range(1,len(bounds)): d += f'L {bounds[i][0]} {bounds[i][1]} '
d += 'Z'
_svg_.append(f'<path d="{d}" fill="none" stroke="#c0c0c0" stroke-width="3.0" />')

_cells_ = rt.laguerreVoronoi(three_circles, bounds)
for i in range(len(_cells_)):
    _cell_ = _cells_[i]
    d = f'M {_cell_[0][0]} {_cell_[0][1]} '
    for j in range(1,len(_cell_)): d += f'L {_cell_[j][0]} {_cell_[j][1]} '
    d += 'Z'
    _svg_.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1.0" />')

_svg_.append('</svg>')
rt.tile([''.join(_svg_)])